In [2]:
import pandas as pd
import os
import numpy as np

In [54]:
path = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
file = "labeled_comments_scores.csv" # previous version: labeled_comments.csv does not have scores

In [55]:
comments_labeled = pd.read_csv(os.path.join(path, file))
comments = pd.read_csv(os.path.join(path, "merged_comments.csv"))
videos = pd.read_csv(os.path.join(path, "videos_101223.csv"))
videos_trends = pd.read_csv(os.path.join(path, "final_yoga_videos_tends_101023.csv"))
trends = pd.read_csv(os.path.join(path, "final_yoga_channels_trends_091523.csv"))
# matching with the creator id
#comments['creator_id'] = comments.merge(videos,how='left',on='video_id').creator_id.values
#comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values

In [35]:
print(videos.columns)

Index(['video_id', 'title', 'creator', 'creator_id', 'publish_date',
       'duration', 'v1', 'v2', 'v3', 'v7', 'v30', 'e1', 'e2', 'e3', 'e7',
       'e30', 'views', 'enagement', 'likes', 'shares', 'comments', 'keywords',
       'topics', 'challenge'],
      dtype='object')


In [36]:
print(videos_trends.head())

      video_id        date   views  engagements
0  CpoaI_rVRXU  2023-09-06  2023.0        100.0
1  CpoaI_rVRXU  2023-09-07   918.0          5.0
2  CpoaI_rVRXU  2023-09-08    45.0          1.0
3  CpoaI_rVRXU  2023-09-09    33.0          0.0
4  CpoaI_rVRXU  2023-09-10    36.0          0.0


In [56]:
print(comments.head())

      video_id                   user_id  \
0  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
1  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
2  YxySUy2jU_s  UCLMqrKUz0vnpsM0haZmHmQA   
3  YxySUy2jU_s  UCASrUw6lnbPK4p5plkVFtqw   
4  YxySUy2jU_s  UCspkSCEGRikNmsoRGJwG1MQ   

                                        comment_text  no_likes  no_replies  \
0  I loved doing this video with my dad I think i...       8.0         0.0   
1             bre is very good at yoga same with Flo      10.0         0.0   
2  Of course I can add 15 more minutes to round o...       7.0         0.0   
3  Thanks for helping round out my routine, was q...       5.0         0.0   
4          First thing tomorrow morning! Thank you 🙏       2.0         0.0   

   comment_year  comment_month  comment_day  
0        2021.0            4.0         13.0  
1        2021.0            4.0         13.0  
2        2021.0            4.0         13.0  
3        2021.0            4.0         13.0  
4        2021.0            4.0      

In [57]:
print(comments_labeled.head())

      video_id                   user_id  \
0  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
1  YxySUy2jU_s  UCLqs5JoktNvfrQZimZ8Gl5g   
2  YxySUy2jU_s  UCLMqrKUz0vnpsM0haZmHmQA   
3  YxySUy2jU_s  UCASrUw6lnbPK4p5plkVFtqw   
4  YxySUy2jU_s  UCspkSCEGRikNmsoRGJwG1MQ   

                                        comment_text  no_likes  no_replies  \
0  I loved doing this video with my dad I think i...       8.0         0.0   
1             bre is very good at yoga same with Flo      10.0         0.0   
2  Of course I can add 15 more minutes to round o...       7.0         0.0   
3  Thanks for helping round out my routine, was q...       5.0         0.0   
4          First thing tomorrow morning! Thank you 🙏       2.0         0.0   

   comment_year  comment_month  comment_day  compentence  interest  \
0        2021.0            4.0         13.0          NaN       NaN   
1        2021.0            4.0         13.0          NaN       NaN   
2        2021.0            4.0         13.0          NaN

In [58]:
# threshold = 11
comments_labeled = comments_labeled[comments_labeled.video_id.str.len() == 11].copy()
print(comments_labeled.shape)
# drop rows with missing year, month, or day
comments_labeled = comments_labeled.dropna(subset=['comment_year', 'comment_month', 'comment_day'])
print(comments_labeled.shape)
print(comments_labeled.isnull().sum())

(827566, 15)
(827552, 15)
video_id              0
user_id               0
comment_text        100
no_likes              0
no_replies            0
comment_year          0
comment_month         0
comment_day           0
compentence      605574
interest         605574
appearance       605574
fitness          605574
social           605574
label            605574
label_score      605574
dtype: int64


In [59]:
# Create a function to format and convert the date
def to_datetime_with_padding(year, month, day):
    # Pad month and day with zeros if necessary
    month = f"{month:02d}"
    day = f"{day:02d}"
    # Combine and convert to datetime
    date_str = f"{year}-{month}-{day}"
    return pd.to_datetime(date_str)

comments_labeled['comment_year'] = comments_labeled.comment_year.astype(int)
comments_labeled['comment_month'] = comments_labeled.comment_month.astype(int)
comments_labeled['comment_day'] = comments_labeled.comment_day.astype(int)

# Apply the function to each row
comments_labeled['date'] = comments_labeled.apply(lambda row: to_datetime_with_padding(row["comment_year"], row["comment_month"], row["comment_day"]), axis=1)

In [91]:
# fix a typo
comments_labeled = comments_labeled.rename(columns= {'compentence': 'competence'})

In [92]:
comments_labeled.fillna(0).to_csv(os.path.join(path, "labeled_comments_scores_edited.csv"), index=False)

In [72]:
# merging with video dataset
#comments_labeled = 
comments_labeled['creator_id'] = comments_labeled.merge(videos,how='left',on='video_id').creator_id.values
comments_labeled = comments_labeled.dropna(subset=['creator_id'])
print(comments_labeled.isnull().sum())

video_id              0
user_id               0
comment_text        100
no_likes              0
no_replies            0
comment_year          0
comment_month         0
comment_day           0
compentence      591418
interest         591418
appearance       591418
fitness          591418
social           591418
label            591418
label_score      591418
date                  0
creator_id            0
dtype: int64


In [87]:
channel_day = comments_labeled.fillna(0).groupby(['creator_id','date']).label.value_counts().unstack().reset_index()
channel_day = channel_day.fillna(0)
channel_day = channel_day.rename(columns={0:'none'})
print(channel_day.shape)
print(channel_day.creator_id.nunique())
print(channel_day.head())


(48673, 8)
71
label           creator_id       date  none  appearance  competence  fitness  \
0      262h9yTG1f03Nobo501 2020-11-05   1.0         0.0         0.0      0.0   
1      262h9yTG1f03Nobo501 2020-11-11  25.0         0.0         3.0      0.0   
2      262h9yTG1f03Nobo501 2020-11-12  37.0         0.0         7.0      1.0   
3      262h9yTG1f03Nobo501 2020-11-13  34.0         0.0         5.0      2.0   
4      262h9yTG1f03Nobo501 2020-11-14  34.0         0.0         4.0      6.0   

label  interest  social  
0           0.0     0.0  
1           0.0     0.0  
2           1.0     0.0  
3           1.0     0.0  
4           2.0     0.0  


In [88]:
channel_day.to_csv(os.path.join(path, "channel_day.csv"), index=False)

In [95]:
# Channel day with scores
cols = ['competence','appearance','fitness','interest','social']
channel_day_scores = comments_labeled.fillna(0).groupby(['creator_id','date'])[cols].sum().reset_index()
channel_day_scores.head()
channel_day_scores['no_comments'] = comments_labeled.groupby(['creator_id','date']).comment_text.count().values
channel_day_scores['no_users'] = comments_labeled.groupby(['creator_id','date']).user_id.nunique().values

print(channel_day_scores.shape)
print(channel_day_scores.head())
print(channel_day_scores.isnull().sum())

channel_day_scores.to_csv(os.path.join(path, "channel_day_scores.csv"), index=False)

(48673, 9)
            creator_id       date  competence  appearance   fitness  interest  \
0  262h9yTG1f03Nobo501 2020-11-05    0.000000         0.0  0.000000  0.000000   
1  262h9yTG1f03Nobo501 2020-11-11    0.445001         0.0  0.000000  0.000000   
2  262h9yTG1f03Nobo501 2020-11-12    0.000000         0.0  0.639770  0.714219   
3  262h9yTG1f03Nobo501 2020-11-13    0.000000         0.0  1.225794  0.548560   
4  262h9yTG1f03Nobo501 2020-11-14    0.000000         0.0  4.965698  1.349038   

   social  no_comments  no_users  
0     0.0            1         1  
1     0.0           28        28  
2     0.0           46        41  
3     0.0           42        36  
4     0.0           46        36  
creator_id     0
date           0
competence     0
appearance     0
fitness        0
interest       0
social         0
no_comments    0
no_users       0
dtype: int64


In [96]:
channel_day = pd.read_csv(os.path.join(path, "channel_day.csv"))
merged = trends.merge(channel_day,how='left',on=['creator_id','date']) #.to_csv(os.path.join(path, "trends_channel_day.csv"), index=False)
merged.isnull().sum()
print(f"number of creators with comments: {merged[merged.appearance.notnull()].creator_id.nunique()}, out of {merged.creator_id.nunique()} creators in trends")
# there seems to be a problem with the data set, the number of creators with comments is less than the number of creators in trends

number of creators with comments: 71, out of 75 creators in trends


## Individual Level Data

In [52]:
comments_labeled.groupby(['user_id','date']).comment_text.count().reset_index().to_csv(os.path.join(path, "user_day.csv"), index=False)

## Summary Statistics

In [105]:
comments_labeled['challenge'] = comments_labeled.merge(videos,how='left',on='video_id').challenge.values
comments_labeled.challenge.value_counts()

challenge
False    602105
True     207216
Name: count, dtype: int64

In [119]:
# ratio of comments for challenge and non-challenge videos
summary = comments_labeled.groupby(['challenge']).label.value_counts().unstack().reset_index()
summary.iloc[:,1:] = summary.iloc[:,1:] / comments_labeled.challenge.value_counts().values.reshape(-1,1)
summary

label,challenge,appearance,competence,fitness,interest,social
0,False,0.003235,0.167793,0.044100,0.021619,0.002129
1,True,0.001650,0.280114,0.040991,0.032107,0.002611
